# Data preparation for the learning phase

Install of the library which contains all data

In [ ]:
pip install fastf1

In [ ]:
import fastf1
import pandas as pd

Getting all data for most recent years with same compounds types

In [ ]:
schedules = []
for year in [2019, 2020, 2021, 2022, 2023]:
#for year in [2023]:
    schedules.append(fastf1.get_event_schedule(year))

## Examples of use
Example of preparation phase on a single race

In [ ]:
race = schedules[0].get_event_by_round(1).get_race()
race.load()
race.laps

In [ ]:
race.laps.columns

In [ ]:
race.laps.loc[:, [False, True, True, True, True, True,
       False, False, False, False, False,
       False, False, False,
       False, False, False, False, False,
       True, True, True, True, False,
       False, False, False, False, False,
       False, False]]

In [ ]:
race.results.columns

In [ ]:
race.results.loc[:, [True, False, True, False, True,
       False, False, False, False, False,
       False, False, True, False,
       True, False, False, False, False, False, False]]

In [ ]:
race.weather_data.columns

In [ ]:
race.weather_data.loc[:, [False, True, True, True, True, True,
       False, True]]

In [ ]:
race.weather_data.loc[:, [False, True, True, True, True, True, False, True]].iloc[0]

## Extraction

Preparing list of colums which have to be selected in the tables of interest:
- Laps table: Driver, DriverNumber, LapNumber, Compound, TyreLife, FreshTyre, Team
- Results table: DriverNumber, Position, GridPosition
- Weather Data table: AirTemp, Humidity, Pressure, Rainfall, TrackTemp, WindSpeed

In [ ]:
laps_list = [False, True, True, False, True, False,
       False, False, False, False, False,
       False, False, False,
       False, False, False, False, False,
       True, True, True, True, False,
       False, False, False, False, False,
       False, False]

results_list = [True, False, False, False, False,
       False, False, False, False, False,
       False, False, True, False,
       True, False, False, False, False, False, False]

weather_list = [False, True, True, True, True, True,
       False, True]

Loading of all the data and extracting only column of interest.

About race laps is extracted for each driver the first lap

About weather conditions is selected only information about the first lap

If the code generate an error **** it is normal because of reached maximum number of request per hour, so try again to extract remaining file or use what it downloaded ;)

In [ ]:
peryear_races = []
peryear_races_results = []
peryear_races_laps = []
peryear_races_weather = []

i = 0
for schedule in schedules:
    peryear_races.append({})
    peryear_races_results.append({})
    peryear_races_laps.append({})
    peryear_races_weather.append({})
    
    for round in range(1, len(schedule)-1):
        peryear_races[i][round] = schedule.get_event_by_round(round).get_race()
        peryear_races[i][round].load()
        peryear_races_results[i][round] = peryear_races[i][round].results.loc[:, results_list]
        peryear_races_laps[i][round] = peryear_races[i][round].laps.loc[:, laps_list].pick_lap(1)
        peryear_races_weather[i][round] = peryear_races[i][round].weather_data.loc[:, weather_list].head()
    
    i+=1
    

Merge results and first lap information and adding of round name and year for each race

Adding for each race weather condition information about round name and year of attendance

For both kind of information is created a directory called './data', where all csv files are saved with the format 'results_year_roundname.csv' and weather_year_roundname.csv'

In [ ]:
merge_lapsresults = []
base_year = 2019
i = 0

results_header= pd.DataFrame(columns=['Driver','DriverNumber','LapNumber','Compound','TyreLife','FreshTyre','Team','Position','GridPosition','Round','Year'])
results_header.to_csv('./data/results.csv')

weather_header=pd.DataFrame(columns=['AirTemp', 'Humidity', 'Pressure', 'Rainfall', 'TrackTemp', 'WindSpeed', 'Round', 'Year'])
weather_header.to_csv('./data/weather.csv')


for schedule in schedules: 
    merge_lapsresults.append({})
      
    for round in range(1, len(schedule)-1):
        round_name = str(peryear_races[i][round]).split(': ')[1].split(' - ')[0]
        year = base_year+i
        merge_lapsresults[i][round] = pd.merge(peryear_races_laps[i][round], peryear_races_results[i][round], on=["DriverNumber"])
        merge_lapsresults[i][round]['Round'] = round_name
        merge_lapsresults[i][round]['Year'] = year
        peryear_races_weather[i][round]['Round'] = round_name
        peryear_races_weather[i][round]['Year'] = year
        
        merge_lapsresults[i][round].to_csv('./data/results.csv', mode='a', header=False)
        peryear_races_weather[i][round].to_csv('./data/weather.csv', mode='a', header=False)

    i+=1